***GENERATED CODE FOR predicthighestincomeapp PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Occupation', 'transformation_label': 'String Indexer'}], 'feature': 'Occupation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '558', 'mean': '', 'stddev': '', 'min': 'AGRICULTURAL', 'max': 'Writers and authors', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Occupation'}, {'feature_label': 'Occupation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Occupation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'M_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '558', 'mean': '1002.69', 'stddev': '398.31', 'min': '1001', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'M_weekly'}, {'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('M_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'F_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '558', 'mean': '805.43', 'stddev': '305.5', 'min': '1002', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'F_weekly'}, {'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('F_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'All_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '558', 'mean': '910.05', 'stddev': '353.26', 'min': '1000', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'All_weekly'}, {'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('All_weekly')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run predicthighestincomeappHooks.ipynb
try:
	#sourcePreExecutionHook()

	predicthighestincometrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1623758453778_Predict Highest Income Train.csv', 'filename': '1623758453778_Predict Highest Income Train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Finance Models/Predict Highest Income-Regression/Predict Highest Income Test.csv', 'viewFileName': 'Predict Highest Income Train.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run predicthighestincomeappHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(predicthighestincometrain,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/1623758453778_Predict Highest Income Train.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/1623758453778_Predict Highest Income Train.csv", "data_source": "platfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/60f186ca5ab8a60e712903d61626517936/0part.csv", "requestRatio": 1.0, "totalRows": 558, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 7, "numberOfRows": 558, "duplicateRowCount": 0, "stats": [{"column": "Occupation", "alias": "Occupation", "generated": 0, "type": "String", "max": "Writers and authors", "min": "AGRICULTURAL", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "M_workers", "alias": "M_workers", "generated": 0, "type": "real", "max": 60746.0, "min": 0.0, "mean": 326.5931899641577, "missing": 0.0, "stddev": 2654.6}, {"column": "M_weekly", "alias": "M_weekly", "generated": 0, "type": "String", "max": "Na", "min": "1001", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "F_workers", "alias": "F_workers", "generated": 0, "type": "real", "max": 48334.0, "min": 0.0, "mean": 259.831541218638, "missing": 0.0, "stddev": 2142.55}, {"column": "F_weekly", "alias": "F_weekly", "generated": 0, "type": "String", "max": "Na", "min": "1002", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "All_workers", "alias": "All_workers", "generated": 0, "type": "real", "max": 109080.0, "min": 0.0, "mean": 586.4587813620071, "missing": 0.0, "stddev": 4758.98}, {"column": "All_weekly", "alias": "All_weekly", "generated": 0, "type": "String", "max": "Na", "min": "1000", "mean": "", "missing": 0.0, "stddev": ""}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "M_weekly", "type": "String", "min": "1001", "max": "Na", "mean": "-"}], "functionsData": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "min", "ReplaceWith": ""}]}], "functionChanges": [{"columnName": "M_weekly", "functionName": "Replace Outliers", "Type": "String", "Parameters": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "min", "ReplaceWith": ""}]}], "fileheader": [{"field": "Occupation", "alias": "Occupation", "generated": 0, "position": 1, "type": "String"}, {"field": "M_workers", "alias": "M_workers", "generated": 0, "position": 2, "type": "real"}, {"field": "M_weekly", "alias": "M_weekly", "generated": 0, "position": 3, "type": "String"}, {"field": "F_workers", "alias": "F_workers", "generated": 0, "position": 4, "type": "real"}, {"field": "F_weekly", "alias": "F_weekly", "generated": 0, "position": 5, "type": "String"}, {"field": "All_workers", "alias": "All_workers", "generated": 0, "position": 6, "type": "real"}, {"field": "All_weekly", "alias": "All_weekly", "generated": 0, "position": 7, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run predicthighestincomeappHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Occupation", "transformation_label": "String Indexer"}], "feature": "Occupation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "558", "mean": "", "stddev": "", "min": "AGRICULTURAL", "max": "Writers and authors", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Occupation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "M_workers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "558", "mean": "326.59", "stddev": "2654.6", "min": "0.0", "max": "60746.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "M_workers"}, {"transformationsData": [{"feature_label": "M_weekly", "transformation_label": "String Indexer"}], "feature": "M_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "558", "mean": "1002.69", "stddev": "398.31", "min": "1001", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "M_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "F_workers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "558", "mean": "259.83", "stddev": "2142.55", "min": "0.0", "max": "48334.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "F_workers"}, {"transformationsData": [{"feature_label": "F_weekly", "transformation_label": "String Indexer"}], "feature": "F_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "558", "mean": "805.43", "stddev": "305.5", "min": "1002", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "F_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "All_workers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "558", "mean": "586.46", "stddev": "4758.98", "min": "0.0", "max": "109080.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "All_workers"}, {"transformationsData": [{"feature_label": "All_weekly", "transformation_label": "String Indexer"}], "feature": "All_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "558", "mean": "910.05", "stddev": "353.26", "min": "1000", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "All_weekly"}, {"feature": "Occupation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "558", "mean": "278.5", "stddev": "161.22", "min": "0.0", "max": "557.0", "missing": "0"}, "updatedLabel": "Occupation_stringindexer_..."}, {"feature": "M_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "558", "mean": "40.5", "stddev": "63.44", "min": "0.0", "max": "211.0", "missing": "0"}, "updatedLabel": "M_weekly_stringindexer_tr..."}, {"feature": "F_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "558", "mean": "29.93", "stddev": "52.35", "min": "0.0", "max": "182.0", "missing": "0"}, "updatedLabel": "F_weekly_stringindexer_tr..."}, {"feature": "All_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "558", "mean": "70.08", "stddev": "88.17", "min": "0.0", "max": "276.0", "missing": "0"}, "updatedLabel": "All_weekly_stringindexer_..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run predicthighestincomeappHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(autofe, ["Occupation_stringindexer", "M_workers", "M_weekly_stringindexer", "F_workers", "F_weekly_stringindexer", "All_workers"], "All_weekly_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
